In [31]:
"""Hello Analytics Reporting API V4."""
#import everything upwards from home dir
from __future__ import absolute_import
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from configparser import ConfigParser
import psycopg2
import argparse
import six
import sys
import google.ads.google_ads.client
import pandas as pd
import re
import numpy as np
import time

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_secrets.json'


VIEW_ID = '6485278'
VIEW_ID = '166655749'
#VIEW_ID = '83024411'
# session count, source / medium, goals


def initialize_analyticsreporting():
    """Initializes an Analytics Reporting API V4 service object.

    Returns:
        An authorized Analytics Reporting API V4 service object.
    """
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
            KEY_FILE_LOCATION, SCOPES)

    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

def db_config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
 
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db

def get_report(analytics, df_conf_base, df_conf_req):
    """Queries the Analytics Reporting API V4.

    Args:
        analytics: An authorized Analytics Reporting API V4 service object.
    Returns:
        The Analytics Reporting API V4 response.
    """
    
    df_response = pd.DataFrame()
    
    dim_lst = []
    met_lst = []
    start_date = df_conf_req.iat[0,2]
    end_date = df_conf_req.iat[1,2]
    for index, req in df_conf_req.iterrows():
        if not 'nan' in str(req['dimensions']):
            dim_lst.append(dict({'name':req['dimensions']}))
        if not 'nan' in str(req['metrics']):
            met_lst.append(dict({'expression':req['metrics']}))  
    
    dim_met_lst = dim_lst +  met_lst
    
    met_batches = list()
    
    i = 0
    while True:
        met_batch = list()
        stop = 0
        for index, met in enumerate(met_lst):
            if len(met_batch) < 9 and i < len(met_lst):
                res = met_batch.append(met_lst[i])
                i=i+1
        if len(met_batch) != 0:
            met_batches.append(met_batch)
        if i == len(met_lst):
            x = True
            break 
            
#     met_batches_fix = {}
#     for batch in met_batches:
#         if "{'expression': 'ga:sessions'}" not in str(batch):
#             print(met_batches)
#             print(met_batches_fix)
#             #batch = dict(batch[0])
#             print(batch)
#             batch = dict(batch[0]).update({'expression' : 'ga:sessions'})
#         met_batches_fix = met_batches_fix.update(batch)
    
    
    
#     print(met_batches)
#     print(met_batches_fix)
    df_res_part = pd.DataFrame()
    #met_batches = met_batches_fix
    for index, batch in enumerate(met_batches):
        #time.sleep(3)
        body1={
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': batch,
            'dimensions': dim_lst,
            
            'pageSize': 10000

        }]
    }    
        #print(batch)
        #print(body1)
        response = analytics.reports().batchGet(body=body1).execute()
        print(response)
        for report in response.get('reports', []):
            columnHeader = report.get('columnHeader', {})
            dimensionHeaders = columnHeader.get('dimensions', [])
            metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
            
#             print(batch)
#             print('shit')
#             print(report.get('data', {}).get('rows', []))
#             print(response)
            
            for data_row in report.get('data', {}).get('rows', []):
                dimensions = data_row.get('dimensions', [])
                dateRangeValues = data_row.get('metrics', [])
                var_dict = {}
#                 print(batch)
#                 print('shit')
                for header, dimension in zip(dimensionHeaders, dimensions):
                    if header not in df_res_part:
                        df_res_part[header] = pd.Series()
                        df_res_part.astype({header: 'object'}).dtypes
                    var_dict.update({header : str(dimension)})
                    #print(var_dict)
                for i, values in enumerate(dateRangeValues):
                    for metricHeader, value in zip(metricHeaders, values.get('values')):
                        if metricHeader.get('name') not in df_res_part.columns:
                            df_res_part[metricHeader.get('name')] = pd.Series()
                        var_dict.update({metricHeader.get('name') : value})
                #print(var_dict)
                #print(var_dict)
                df_res_part = df_res_part.append(var_dict, ignore_index=True)
        #print(df_res_part)
        row_count_part = len(df_res_part.index)
        #print('DF_part_row_count: ' + str(row_count_part))
    if df_response.empty:
        df_response = df_res_part
    else:
        df_response = pd.merge(df_response, df_res_part,  how='left', left_on=['ga:campaign', 'ga:adcontent', 'ga:channelGrouping', 'ga:keyword', 'ga:date'], right_on = ['ga:campaign', 'ga:adcontent', 'ga:channelGrouping', 'ga:keyword', 'ga:date'])
    #new_df = pd.merge(A_df, B_df,  how='left', left_on=['A_c1','c2'], right_on = ['B_c1','c2'])
    row_count_full = len(df_response.index)
    #print('DF_full_row_count: ' + str(row_count_full))
    return df_response, dim_met_lst
    

if __name__ == '__main__':
    df_conf_base = pd.read_excel('google_analytics_conf_1.xlsx', sheet_name='base', header=0)
    df_conf_req = pd.read_excel('google_analytics_conf_1.xlsx', sheet_name='parameters', header=0)                    
    analytics = initialize_analyticsreporting()
    google_analytics_response = get_report(analytics, df_conf_base, df_conf_req)
    df_response = google_analytics_response[0]
    dim_met_lst = google_analytics_response[1]
    db_config(filename = 'database.ini')
    postgre_write(df_response, dim_met_lst)

{'reports': [{'columnHeader': {'dimensions': ['ga:sourceMedium', 'ga:campaign', 'ga:adcontent', 'ga:channelGrouping', 'ga:keyword', 'ga:date'], 'metricHeader': {'metricHeaderEntries': [{'name': 'ga:sessions', 'type': 'INTEGER'}, {'name': 'ga:transactions', 'type': 'INTEGER'}, {'name': 'ga:transactionRevenue', 'type': 'CURRENCY'}, {'name': 'ga:goal1Value', 'type': 'CURRENCY'}, {'name': 'ga:goal2Value', 'type': 'CURRENCY'}, {'name': 'ga:goal3Value', 'type': 'CURRENCY'}, {'name': 'ga:goal4Value', 'type': 'CURRENCY'}, {'name': 'ga:goal5Value', 'type': 'CURRENCY'}, {'name': 'ga:goal6Value', 'type': 'CURRENCY'}]}}, 'data': {'rows': [{'dimensions': ['(direct) / (none)', '(not set)', '(not set)', 'Direct', '(not set)', '20191013'], 'metrics': [{'values': ['2490', '0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0']}]}, {'dimensions': ['15min.lt / referral', '(not set)', '(not set)', 'Referral', '(not set)', '20191013'], 'metrics': [{'values': ['2', '0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0

{'reports': [{'columnHeader': {'dimensions': ['ga:sourceMedium', 'ga:campaign', 'ga:adcontent', 'ga:channelGrouping', 'ga:keyword', 'ga:date'], 'metricHeader': {'metricHeaderEntries': [{'name': 'ga:goal7Value', 'type': 'CURRENCY'}, {'name': 'ga:goal8Value', 'type': 'CURRENCY'}, {'name': 'ga:goal9Value', 'type': 'CURRENCY'}, {'name': 'ga:goal10Value', 'type': 'CURRENCY'}, {'name': 'ga:goal11Value', 'type': 'CURRENCY'}, {'name': 'ga:goal12Value', 'type': 'CURRENCY'}, {'name': 'ga:goal13Value', 'type': 'CURRENCY'}, {'name': 'ga:goal14Value', 'type': 'CURRENCY'}, {'name': 'ga:goal15Value', 'type': 'CURRENCY'}]}}, 'data': {'totals': [{'values': ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0']}], 'isDataGolden': True}}]}
{'reports': [{'columnHeader': {'dimensions': ['ga:sourceMedium', 'ga:campaign', 'ga:adcontent', 'ga:channelGrouping', 'ga:keyword', 'ga:date'], 'metricHeader': {'metricHeaderEntries': [{'name': 'ga:goal16Value', 'type': 'CURRENCY'}, {'name': 'ga:goal17Value', 

{'reports': [{'columnHeader': {'dimensions': ['ga:sourceMedium', 'ga:campaign', 'ga:adcontent', 'ga:channelGrouping', 'ga:keyword', 'ga:date'], 'metricHeader': {'metricHeaderEntries': [{'name': 'ga:goal5Completions', 'type': 'INTEGER'}, {'name': 'ga:goal6Completions', 'type': 'INTEGER'}, {'name': 'ga:goal7Completions', 'type': 'INTEGER'}, {'name': 'ga:goal8Completions', 'type': 'INTEGER'}, {'name': 'ga:goal9Completions', 'type': 'INTEGER'}, {'name': 'ga:goal10Completions', 'type': 'INTEGER'}, {'name': 'ga:goal11Completions', 'type': 'INTEGER'}, {'name': 'ga:goal12Completions', 'type': 'INTEGER'}, {'name': 'ga:goal13Completions', 'type': 'INTEGER'}]}}, 'data': {'totals': [{'values': ['0', '0', '0', '0', '0', '0', '0', '0', '0']}], 'isDataGolden': True}}]}
{'reports': [{'columnHeader': {'dimensions': ['ga:sourceMedium', 'ga:campaign', 'ga:adcontent', 'ga:channelGrouping', 'ga:keyword', 'ga:date'], 'metricHeader': {'metricHeaderEntries': [{'name': 'ga:goal14Completions', 'type': 'INTEGER'

ga:keyword
ga:date
ga:sessions
ga:transactions
ga:transactionRevenue
ga:goal1Value
ga:goal2Value
ga:goal3Value
ga:goal4Value
ga:goal5Value
ga:goal6Value
ga:goal16Value
ga:goal17Value
ga:goal18Value
ga:goal19Value
ga:goal20Value
ga:goal1Completions
ga:goal2Completions
ga:goal3Completions
ga:goal4Completions
INSERT INTO google_analytics_temp (ga_sourceMedium,ga_campaign,ga_adcontent,ga_channelGrouping,ga_keyword,ga_date,ga_sessions,ga_transactions,ga_transactionRevenue,ga_goal1Value,ga_goal2Value,ga_goal3Value,ga_goal4Value,ga_goal5Value,ga_goal6Value) VALUES ( 'away.vk.com / referral', '(not set)', '(not set)', 'Social', '(not set)', '20191013', '2', '0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0') ON CONFLICT (ga_sourceMedium, ga_date, ga_campaign, ga_adcontent, ga_channelGrouping, ga_keyword) DO UPDATE SET ga_sessions = 2, ga_transactions = 0, ga_transactionRevenue = 0.0, ga_goal1Value = 0.0, ga_goal2Value = 0.0, ga_goal3Value = 0.0, ga_goal4Value = 0.0, ga_goal5Value = 0.0, ga_g

ga:date
ga:sessions
ga:transactions
ga:transactionRevenue
ga:goal1Value
ga:goal2Value
ga:goal3Value
ga:goal4Value
ga:goal5Value
ga:goal6Value
ga:goal16Value
ga:goal17Value
ga:goal18Value
ga:goal19Value
ga:goal20Value
ga:goal1Completions
ga:goal2Completions
ga:goal3Completions
ga:goal4Completions
INSERT INTO google_analytics_temp (ga_sourceMedium,ga_campaign,ga_adcontent,ga_channelGrouping,ga_keyword,ga_date,ga_sessions,ga_transactions,ga_transactionRevenue,ga_goal1Value,ga_goal2Value,ga_goal3Value,ga_goal4Value,ga_goal5Value,ga_goal6Value) VALUES ( 'bing / organic', '(not set)', '(not set)', 'Organic Search', 'barbo', '20191013', '1', '0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0') ON CONFLICT (ga_sourceMedium, ga_date, ga_campaign, ga_adcontent, ga_channelGrouping, ga_keyword) DO UPDATE SET ga_sessions = 1, ga_transactions = 0, ga_transactionRevenue = 0.0, ga_goal1Value = 0.0, ga_goal2Value = 0.0, ga_goal3Value = 0.0, ga_goal4Value = 0.0, ga_goal5Value = 0.0, ga_goal6Value = 0.0

...

ga:goal3Completions
ga:goal4Completions
INSERT INTO google_analytics_temp (ga_sourceMedium,ga_campaign,ga_adcontent,ga_channelGrouping,ga_keyword,ga_date,ga_goal16Value,ga_goal17Value,ga_goal18Value,ga_goal19Value,ga_goal20Value,ga_goal1Completions,ga_goal2Completions,ga_goal3Completions,ga_goal4Completions) VALUES ( 'lavasoft.gosearchresults.com / referral', '(not set)', '(not set)', 'Referral', '(not set)', '20191013', '0.0', '0.0', '0.0', '0.0', '0.0', '1', '0', '0', '0') ON CONFLICT (ga_sourceMedium, ga_date, ga_campaign, ga_adcontent, ga_channelGrouping, ga_keyword) DO UPDATE SET ga_goal16Value = 0.0, ga_goal17Value = 0.0, ga_goal18Value = 0.0, ga_goal19Value = 0.0, ga_goal20Value = 0.0, ga_goal1Completions = 1, ga_goal2Completions = 0, ga_goal3Completions = 0, ga_goal4Completions = 0; 
Index(['ga:sourceMedium', 'ga:campaign', 'ga:adcontent', 'ga:channelGrouping',
       'ga:keyword', 'ga:date', 'ga:sessions', 'ga:transactions',
       'ga:transactionRevenue', 'ga:goal1Value', 'ga

Index(['ga:sourceMedium', 'ga:campaign', 'ga:adcontent', 'ga:channelGrouping',
       'ga:keyword', 'ga:date', 'ga:sessions', 'ga:transactions',
       'ga:transactionRevenue', 'ga:goal1Value', 'ga:goal2Value',
       'ga:goal3Value', 'ga:goal4Value', 'ga:goal5Value', 'ga:goal6Value',
       'ga:goal16Value', 'ga:goal17Value', 'ga:goal18Value', 'ga:goal19Value',
       'ga:goal20Value', 'ga:goal1Completions', 'ga:goal2Completions',
       'ga:goal3Completions', 'ga:goal4Completions'],
      dtype='object')
ga:sourceMedium
ga:campaign
ga:adcontent
ga:channelGrouping
ga:keyword
ga:date
ga:sessions
ga:transactions
ga:transactionRevenue
ga:goal1Value
ga:goal2Value
ga:goal3Value
ga:goal4Value
ga:goal5Value
ga:goal6Value
ga:goal16Value
ga:goal17Value
ga:goal18Value
ga:goal19Value
ga:goal20Value
ga:goal1Completions
ga:goal2Completions
ga:goal3Completions
ga:goal4Completions
INSERT INTO google_analytics_temp (ga_sourceMedium,ga_campaign,ga_adcontent,ga_channelGrouping,ga_keyword,ga_date,ga_goa

ga:campaign
ga:adcontent
ga:channelGrouping
ga:keyword
ga:date
ga:sessions
ga:transactions
ga:transactionRevenue
ga:goal1Value
ga:goal2Value
ga:goal3Value
ga:goal4Value
ga:goal5Value
ga:goal6Value
ga:goal16Value
ga:goal17Value
ga:goal18Value
ga:goal19Value
ga:goal20Value
ga:goal1Completions
ga:goal2Completions
ga:goal3Completions
ga:goal4Completions
INSERT INTO google_analytics_temp (ga_sourceMedium,ga_campaign,ga_adcontent,ga_channelGrouping,ga_keyword,ga_date,ga_goal16Value,ga_goal17Value,ga_goal18Value,ga_goal19Value,ga_goal20Value,ga_goal1Completions,ga_goal2Completions,ga_goal3Completions,ga_goal4Completions) VALUES ( 'newsletter / email', 'http_uns_char_fnd_ascii_dec_expr_58//barbora.lt', '(not set)', 'Email', '(not set)', '20191013', '0.0', '0.0', '0.0', '0.0', '0.0', '29', '0', '0', '0') ON CONFLICT (ga_sourceMedium, ga_date, ga_campaign, ga_adcontent, ga_channelGrouping, ga_keyword) DO UPDATE SET ga_goal16Value = 0.0, ga_goal17Value = 0.0, ga_goal18Value = 0.0, ga_goal19Value 

ga:keyword
ga:date
ga:sessions
ga:transactions
ga:transactionRevenue
ga:goal1Value
ga:goal2Value
ga:goal3Value
ga:goal4Value
ga:goal5Value
ga:goal6Value
ga:goal16Value
ga:goal17Value
ga:goal18Value
ga:goal19Value
ga:goal20Value
ga:goal1Completions
ga:goal2Completions
ga:goal3Completions
ga:goal4Completions
INSERT INTO google_analytics_temp (ga_sourceMedium,ga_campaign,ga_adcontent,ga_channelGrouping,ga_keyword,ga_date,ga_goal16Value,ga_goal17Value,ga_goal18Value,ga_goal19Value,ga_goal20Value,ga_goal1Completions,ga_goal2Completions,ga_goal3Completions,ga_goal4Completions) VALUES ( 'r.send.tellq.io / referral', '(not set)', '(not set)', 'Referral', '(not set)', '20191013', '0.0', '0.0', '0.0', '0.0', '0.0', '2', '0', '0', '0') ON CONFLICT (ga_sourceMedium, ga_date, ga_campaign, ga_adcontent, ga_channelGrouping, ga_keyword) DO UPDATE SET ga_goal16Value = 0.0, ga_goal17Value = 0.0, ga_goal18Value = 0.0, ga_goal19Value = 0.0, ga_goal20Value = 0.0, ga_goal1Completions = 2, ga_goal2Completion

ga:goal17Value
ga:goal18Value
ga:goal19Value
ga:goal20Value
ga:goal1Completions
ga:goal2Completions
ga:goal3Completions
ga:goal4Completions
INSERT INTO google_analytics_temp (ga_sourceMedium,ga_campaign,ga_adcontent,ga_channelGrouping,ga_keyword,ga_date,ga_goal16Value,ga_goal17Value,ga_goal18Value,ga_goal19Value,ga_goal20Value,ga_goal1Completions,ga_goal2Completions,ga_goal3Completions,ga_goal4Completions) VALUES ( 'yahoo / organic', '(not set)', '(not set)', 'Organic Search', '(not provided)', '20191013', '0.0', '0.0', '0.0', '0.0', '0.0', '3', '0', '0', '0') ON CONFLICT (ga_sourceMedium, ga_date, ga_campaign, ga_adcontent, ga_channelGrouping, ga_keyword) DO UPDATE SET ga_goal16Value = 0.0, ga_goal17Value = 0.0, ga_goal18Value = 0.0, ga_goal19Value = 0.0, ga_goal20Value = 0.0, ga_goal1Completions = 3, ga_goal2Completions = 0, ga_goal3Completions = 0, ga_goal4Completions = 0; 
Index(['ga:sourceMedium', 'ga:campaign', 'ga:adcontent', 'ga:channelGrouping',
       'ga:keyword', 'ga:date',

In [30]:
def db_config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
 
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db

def postgre_write(df_response, dim_lst):
    #Connect to the PostgreSQL database server
    conn = None
    # read connection parameters
    params = db_config()
    
    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(**params)
 
    try:
        # create a cursor
        cur = conn.cursor()

       # execute db write statements
        print('PostgreSQL database version:')
        cur.execute("DROP TABLE google_analytics_temp;")
        cur.execute("CREATE TABLE IF NOT EXISTS google_analytics_temp("
                       "ga_sourceMedium VARCHAR(200), "
                       "ga_date DATE, "
                       "ga_campaign VARCHAR(150), "
                       "ga_adcontent VARCHAR(150), "
                       "ga_channelGrouping VARCHAR(150), "
                       "ga_keyword VARCHAR(150), "
                       "CONSTRAINT table_analytics_pk PRIMARY KEY (ga_sourceMedium, ga_date, ga_campaign, ga_adcontent, ga_channelGrouping, ga_keyword));")
        conn.commit()    
        
        print(df_response.dtypes)
        
        for i, row in df_response.iterrows():
            ins_query_dim =''
            ins_query_val = ''
            upd_query = ''
            col_dtype = ''
            print(row.index)
            for idx, dim in enumerate(list(row.index)):
                db_dim = dim
                print(dim)
                val = str(row[dim])
                uns_char_lst = ["'", ":"]
                for char in uns_char_lst:
                    if char in str(row[dim]):
                        val = val.replace(char, "_uns_char_fnd_ascii_dec_expr_" + str(ord(char)))
                    if char in db_dim:
                        db_dim = db_dim.replace(char, '_')
                if str(row[dim]) != 'nan':
                    #val = 'NULL'
                    if str(db_dim) == 'ga_sessions':
                        col_dtype = 'bigint'
                        cur.execute(f"ALTER TABLE google_analytics_temp "
                                    f"ADD COLUMN IF NOT EXISTS {db_dim} {col_dtype};")
                        ins_query_dim = ins_query_dim + db_dim + ','
                        ins_query_val = ins_query_val + " '" + val + "',"
                        upd_query = upd_query + db_dim + ' = ' + val + ', '
                        
                    elif 'goal' in str(db_dim) or 'transaction' in str(db_dim):
                        col_dtype = 'real'
                        cur.execute(f"ALTER TABLE google_analytics_temp "
                                    f"ADD COLUMN IF NOT EXISTS {db_dim} {col_dtype};")
                        ins_query_dim = ins_query_dim + db_dim + ','
                        ins_query_val = ins_query_val + " '" + val + "',"
                        upd_query = upd_query + db_dim + ' = ' + val + ', '
                    else:
                        col_dtype = 'varchar (150)'
                        cur.execute(f"ALTER TABLE google_analytics_temp "
                                    f"ADD COLUMN IF NOT EXISTS {db_dim} {col_dtype};")
                        ins_query_dim = ins_query_dim + db_dim + ','
                        ins_query_val = ins_query_val + " '" + val + "',"
                        #upd_query = upd_query + db_dim + ' = ' + val + ', '
            ins_query_dim = ins_query_dim[:-1]
            ins_query_val = ins_query_val[:-1]
            upd_query = upd_query[:-2]

            print(f"INSERT INTO google_analytics_temp ({ins_query_dim}) "
                        f"VALUES ({ins_query_val}) "
                        f"ON CONFLICT (ga_sourceMedium, ga_date, ga_campaign, ga_adcontent, ga_channelGrouping, ga_keyword) " 
                            f"DO "
                                f"UPDATE "
                                f"SET {upd_query}; ")
            cur.execute(f"INSERT INTO google_analytics_temp ({ins_query_dim}) "
                        f"VALUES ({ins_query_val}) "
                        f"ON CONFLICT (ga_sourceMedium, ga_date, ga_campaign, ga_adcontent, ga_channelGrouping, ga_keyword) " 
                            f"DO "
                                f"UPDATE "
                                f"SET {upd_query}; ")
            conn.commit()
       
       # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    print('Database connection closed.')

In [26]:
import json
data = json.load(response)

AttributeError: 'dict' object has no attribute 'read'

In [56]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
response

SyntaxError: invalid syntax (<ipython-input-56-d3bfc748244a>, line 1)

In [181]:
shit = {'expression': 'ga:goal20Completions'}

In [189]:
ass = list(shit.values())[0]

In [190]:
print(ass)

ga:goal20Completions


In [23]:
ord("'")

39